# http://tractometer.org/ismrm_2015_challenge
## http://tractometer.org/downloads/downloads/ISMRM_2015_Tracto_challenge_data_v1_1.zip

## Generate streamlines

In [ ]:
from os.path import expanduser, join

In [ ]:
home = '/home/favreau/medias/dti/'
dname = join(home, 'ISMRM_2015_Tracto_challenge_data')
fraw = join(dname, 'Diffusion.nii.gz')
fbval = join(dname, 'Diffusion.bvals')
fbvec = join(dname, 'Diffusion.bvecs')

In [ ]:
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table
import nibabel as nib

bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals, bvecs)
img = nib.load(fraw)
data = img.get_data()
affine = img.affine

''' '''
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model

csa_model = CsaOdfModel(gtab, sh_order=6)
csa_peaks = peaks_from_model(csa_model, data, default_sphere,
                             relative_peak_threshold=.8,
                             min_separation_angle=45)

''' '''
from dipy.tracking.local import ThresholdTissueClassifier
classifier = ThresholdTissueClassifier(csa_peaks.gfa, .25)

In [ ]:
''' '''
from dipy.tracking import utils

seed_mask = True #labels == 2
seeds = utils.seeds_from_mask(seed_mask, density=[2, 2, 2], affine=affine)

''' '''
from dipy.tracking.local import LocalTracking
from dipy.tracking.streamline import Streamlines

streamlines_generator = LocalTracking(csa_peaks, classifier, seeds, affine, step_size=.5)

# Generate streamlines object
streamlines = Streamlines(streamlines_generator)

In [ ]:
len(streamlines)

# Visualize in Brayns

In [ ]:
import viztools
brayns = viztools.Brayns('localhost:5000')

In [ ]:
def add_streamlines(name, streams, radius, opacity, limit, range = (0, 1e6)):
    ''' Build streamlines'''
    count = 0
    indices = []
    points = []
    for stream in streams[range[0]:range[1]]:
        indices.append(count)
        l = min(len(stream), limit)
        for x in stream[:limit]:
            for z in x:
                    points.append(float(z))
        count = count + l
        
    ''' RPC call to Brayns '''
    params = dict()
    params['name'] = name
    params['indices'] = indices
    params['points'] = points
    params['radius'] = radius
    params['opacity'] = opacity
    result = brayns.rpc_request("streamline", params=params)

In [ ]:
streams_per_model = 1000
for i in range(1, len(streamlines), streams_per_model):
    add_streamlines(str(i) + '-' + str(i + streams_per_model - 1), streamlines, 0.1, 1, 10000, (i, i + streams_per_model - 1))

## Tracks

In [ ]:
folder = home + 'ISMRM_2015_Tracto_challenge_ground_truth_bundles_TRK_v2/'
# folder = '/home/favreau/Dropbox/Python/notebooks/DTI/'
import os, fnmatch
from ipywidgets import widgets
files = fnmatch.filter(os.listdir(folder), '*.trk')
files.sort()
print(files)

In [ ]:
# Load all tracks
from dipy.io.streamline import load_trk
for file in files:
    streams, hdr = load_trk(folder + file)
    add_streamlines(file, streams, 0.1, 1, 100000, (0, len(streams)))

In [ ]:
file = files[0]
streams, hdr = load_trk(folder + file)
add_streamlines(file, streams, 0.01, 1, 100000, (0, len(streams)))

## Movie

### CC

In [ ]:
import math
r = 150
for i in range(360):
    angle = i * math.pi / 180.0
    t = [34.35649871826172, 35.56396484375, 81.80602264404297]
    o = [t[0] + r * math.cos(angle), t[1] + r * math.sin(angle), t[2]]
    brayns.set_camera(origin=o, look_at=t, up=[0,0,1])
    filename = '/home/favreau/medias/movies/dti/%06d.png' % i
#     brayns.image(size=(1920,1080), samples_per_pixel=8, quality=100, format='png').save(filename)

In [ ]:
brayns.camera.look_at

In [ ]:
file = files[0]
print(file)
streams, hdr = load_trk(folder + file)
for i in range(2, 1000, 1):
    add_streamlines(file, streams, 0.1, 1, i, (0, len(streams)))
    filename = '/home/favreau/medias/movies/dti/growing/%06d.png' % i
    brayns.image(size=(1920,1080), samples_per_pixel=8, quality=100, format='png').save(filename)

In [ ]:
brayns.set_camera(stereo_mode=brayns.STEREO_MODE_NONE)